# Phase 1: Protocol Validation for Brain Templates¶

This notebook contains results validating the AFID protocol on five openly available macaque templates (D99, INIA19, MNI, NMTv1.3, and Yerkes19). The protocol and specific instructions for placing the templates were finalized on a consensus basis among raters and participants of the afids-macaca project on BrainWeb (thus Phase 1).

At this point, we'd like to be more sensitive to discrepancies in rater placements so an outlier is defined as a value of > 1.5 mm (3 voxels) from the mean.

The first step is to initialize the variables, define useful functions, and load all the raw fcsv data into df_raters.


In [1]:
# initialize libraries
library(plyr)
library(digest)
library(reshape2)
library(ggplot2)
#library("plot3D")

Registered S3 methods overwritten by 'ggplot2':
  method         from 
  [.quosures     rlang
  c.quosures     rlang
  print.quosures rlang


In [2]:
# useful functions

# calculate the distance between two sets of coordinates
dist3D <- function(coord1, coord2) { # vector X,Y,Z
        xdist <- coord1[1] - coord2[1] # could also write as coord1$X, etc.
        ydist <- coord1[2] - coord2[2]
        zdist <- coord1[3] - coord2[3]
        return(as.numeric(sqrt(xdist^2+ydist^2+zdist^2)))
}

# calculate the pairwise distance between an array of 3D coordinates
pairwise_dist3D <- function(temp_coords) { # labeled X,Y,Z
        N <- length(temp_coords$X)
        dist_vec <- rep(0,N) # create vector
        sum_dist <- 0 # initialize to zero
        count <- 0
        for (i in 1:(N-1)) {
                for (j in (i+1):N) {
                        if (i != j) {
                                count <- count + 1
                                first_coord <- temp_coords[i,]
                                second_coord <- temp_coords[j,]
                                curr_dist <- dist3D(first_coord, second_coord)
                                sum_dist <- sum_dist + curr_dist
                                dist_vec[count] <- curr_dist
                        }
                }
        }
        return(c(as.numeric(mean(dist_vec)),as.numeric(sd(dist_vec))))
}

In [3]:
# initialize variables and load in raw fcsv data into df_raters
setwd('~/Documents/GitHub/afids-macaca/data/PHASE1_input_afid/')

df_afids <- read.table('~/Documents/GitHub/afids-macaca/etc/afids.csv', sep=",", header=TRUE)

df_raters <- data.frame(fid=integer(),X=double(),Y=double(),Z=double(),rater=factor(),
                        template=factor(),mri_type=factor(),session=integer(),
                        name=character(),description=character(),stringsAsFactors = FALSE)
csv_files <- list.files('.', "*.fcsv")

for (i in 1:length(csv_files)) {
    curr_split <- unlist(strsplit(csv_files[i],"_"))
    if (length(curr_split)>1) { # extract name and session data
        rater_template <- curr_split[2]
        rater_mri_type <- curr_split[3]
        rater_name <- curr_split[4]
        rater_session <- as.numeric(unlist(strsplit(curr_split[5],"[.]"))[1])
    }
    curr_rater <- read.table(csv_files[i], header=FALSE, sep=",")
    df_rater <- data.frame(fid = 1:length(curr_rater$V1))

    df_rater <- cbind(df_rater,X=curr_rater[2],Y=curr_rater[3],Z=curr_rater[4],rater=rater_name,
                    template=rater_template,mri_type=rater_mri_type,
                    session=rater_session,name=curr_rater[12],
                    description=curr_rater[13])
  
    df_rater <- rename(df_rater, c("V2"="X","V3"="Y","V4"="Z","V12"="name","V13"="description"))
    df_raters <- rbind(df_raters,df_rater)
}

levels(df_raters$rater) <- 1:8

# Template Averages

For each template, we calculate the mean value for each afid point and store it in a separate .fcsv file so that it can be loaded back into 3D Slicer.

Deviation of the values by > 1.5 mm will be classified as an outlier.

In [4]:
# start by calculating mean coordinates
df_template_mean <- data.frame(fid=integer(),X=double(),Y=double(),Z=double(),
                        template=factor(), name=factor(),description=character(),stringsAsFactors = FALSE)
df_template_sd <- data.frame(fid=integer(),X=double(),Y=double(),Z=double(),
                        template=factor(), name=factor(),description=character(),stringsAsFactors = FALSE)

# iterate over each template and compute the mean and standard deviation
for (curr_template in levels(df_raters$template)) {
    for (i in 1:32) { # for each AFID32 point, calculate the mean
        df_subset <- subset(df_raters, fid == i & template == curr_template)
        curr_fid_name <- df_afids$name[i]
        curr_fid_desc <- df_afids$description[i]
        df_curr_fid <- data.frame(fid = i, X = mean(df_subset$X), Y = mean(df_subset$Y), Z = mean(df_subset$Z),
                        template=curr_template, name=curr_fid_name, description=curr_fid_desc)
        df_template_mean <- rbind(df_template_mean, df_curr_fid)
        df_curr_fid_sd <- data.frame(fid = i, X = sd(df_subset$X), Y = sd(df_subset$Y), Z = sd(df_subset$Z),
                        template=curr_template, name=curr_fid_name, description=curr_fid_desc)
        df_template_sd <- rbind(df_template_sd, df_curr_fid_sd)
    }
}

In [5]:
# Create output fcsv file for each included template

##########################################################################################
# d99
##########################################################################################
df_d99_mean <- subset(df_template_mean, template == 'd99')
df_d99_fcsv <- data.frame(id=paste('vtkMRMLMarkupsFiducialNode',df_d99_mean$fid,sep="_"),x=df_d99_mean$X,y=df_d99_mean$Y,z=df_d99_mean$Z,
                                ow=0,ox=0,oy=0,oz=1,
                                vis=1,sel=1,lock=0,label=df_d99_mean$fid,desc=df_d99_mean$description,
                                associatedNodeID='vtkMRMLScalarVolumeNode1',stringsAsFactors = FALSE)

# write out table (need to use file connection approach because of multiple header lines required by Slicer)
fio <- file('~/Documents/GitHub/afids-macaca/data/PHASE1_output_afid/d99_MEAN.fcsv', open="wt")
writeLines(paste('# Markups fiducial file version = 4.6'),fio)
writeLines(paste('# CoordinateSystem = 0'),fio)
writeLines(paste('# columns = id,x,y,z,ow,ox,oy,oz,vis,sel,lock,label,desc,associatedNodeID'),fio)
write.table(df_d99_fcsv,fio,sep=',',quote=FALSE,col.names=FALSE,row.names=FALSE)
close(fio)

##########################################################################################
# INIA19
##########################################################################################
df_inia19_mean <- subset(df_template_mean, template == 'inia19')
df_inia19_fcsv <- data.frame(id=paste('vtkMRMLMarkupsFiducialNode',df_inia19_mean$fid,sep="_"),x=df_inia19_mean$X,y=df_inia19_mean$Y,z=df_inia19_mean$Z,
                                ow=0,ox=0,oy=0,oz=1,
                                vis=1,sel=1,lock=0,label=df_inia19_mean$fid,desc=df_inia19_mean$description,
                                associatedNodeID='vtkMRMLScalarVolumeNode1',stringsAsFactors = FALSE)

# write out table (need to use file connection approach because of multiple header lines required by Slicer)
fio <- file('~/Documents/GitHub/afids-macaca/data/PHASE1_output_afid/inia19_MEAN.fcsv', open="wt")
writeLines(paste('# Markups fiducial file version = 4.6'),fio)
writeLines(paste('# CoordinateSystem = 0'),fio)
writeLines(paste('# columns = id,x,y,z,ow,ox,oy,oz,vis,sel,lock,label,desc,associatedNodeID'),fio)
write.table(df_inia19_fcsv,fio,sep=',',quote=FALSE,col.names=FALSE,row.names=FALSE)
close(fio)

##########################################################################################
# macaqueMNI
##########################################################################################
df_macaqueMNI_mean <- subset(df_template_mean, template == 'macaqueMNI')
df_macaqueMNI_fcsv <- data.frame(id=paste('vtkMRMLMarkupsFiducialNode',df_macaqueMNI_mean$fid,sep="_"),x=df_macaqueMNI_mean$X,y=df_macaqueMNI_mean$Y,z=df_macaqueMNI_mean$Z,
                                ow=0,ox=0,oy=0,oz=1,
                                vis=1,sel=1,lock=0,label=df_macaqueMNI_mean$fid,desc=df_macaqueMNI_mean$description,
                                associatedNodeID='vtkMRMLScalarVolumeNode1',stringsAsFactors = FALSE)

# write out table (need to use file connection approach because of multiple header lines required by Slicer)
fio <- file('~/Documents/GitHub/afids-macaca/data/PHASE1_output_afid/macaqueMNI_MEAN.fcsv', open="wt")
writeLines(paste('# Markups fiducial file version = 4.6'),fio)
writeLines(paste('# CoordinateSystem = 0'),fio)
writeLines(paste('# columns = id,x,y,z,ow,ox,oy,oz,vis,sel,lock,label,desc,associatedNodeID'),fio)
write.table(df_macaqueMNI_fcsv,fio,sep=',',quote=FALSE,col.names=FALSE,row.names=FALSE)
close(fio)

##########################################################################################
# nmtv1.3
##########################################################################################
df_nmtv1.3_mean <- subset(df_template_mean, template == 'nmtv1.3')
df_nmtv1.3_fcsv <- data.frame(id=paste('vtkMRMLMarkupsFiducialNode',df_nmtv1.3_mean$fid,sep="_"),x=df_nmtv1.3_mean$X,y=df_nmtv1.3_mean$Y,z=df_nmtv1.3_mean$Z,
                                ow=0,ox=0,oy=0,oz=1,
                                vis=1,sel=1,lock=0,label=df_nmtv1.3_mean$fid,desc=df_nmtv1.3_mean$description,
                                associatedNodeID='vtkMRMLScalarVolumeNode1',stringsAsFactors = FALSE)

# write out table (need to use file connection approach because of multiple header lines required by Slicer)
fio <- file('~/Documents/GitHub/afids-macaca/data/PHASE1_output_afid/nmtv1.3_MEAN.fcsv', open="wt")
writeLines(paste('# Markups fiducial file version = 4.6'),fio)
writeLines(paste('# CoordinateSystem = 0'),fio)
writeLines(paste('# columns = id,x,y,z,ow,ox,oy,oz,vis,sel,lock,label,desc,associatedNodeID'),fio)
write.table(df_nmtv1.3_fcsv,fio,sep=',',quote=FALSE,col.names=FALSE,row.names=FALSE)
close(fio)

##########################################################################################
# yerkes19
##########################################################################################
df_yerkes19_mean <- subset(df_template_mean, template == 'yerkes19')
df_yerkes19_fcsv <- data.frame(id=paste('vtkMRMLMarkupsFiducialNode',df_yerkes19_mean$fid,sep="_"),x=df_yerkes19_mean$X,y=df_yerkes19_mean$Y,z=df_yerkes19_mean$Z,
                                ow=0,ox=0,oy=0,oz=1,
                                vis=1,sel=1,lock=0,label=df_yerkes19_mean$fid,desc=df_yerkes19_mean$description,
                                associatedNodeID='vtkMRMLScalarVolumeNode1',stringsAsFactors = FALSE)

# write out table (need to use file connection approach because of multiple header lines required by Slicer)
fio <- file('~/Documents/GitHub/afids-macaca/data/PHASE1_output_afid/yerkes19_MEAN.fcsv', open="wt")
writeLines(paste('# Markups fiducial file version = 4.6'),fio)
writeLines(paste('# CoordinateSystem = 0'),fio)
writeLines(paste('# columns = id,x,y,z,ow,ox,oy,oz,vis,sel,lock,label,desc,associatedNodeID'),fio)
write.table(df_yerkes19_fcsv,fio,sep=',',quote=FALSE,col.names=FALSE,row.names=FALSE)
close(fio)

# Phase 0: Raw Data Analysis

Also classify extreme outliers, defined as >= 1.5 mm from the group mean

In [6]:
df_raters$mean_AFLE <- NA # mean AFID localization error
df_raters$outlier <- NA
df_raters$xdist <- NA
df_raters$ydist <- NA
df_raters$zdist <- NA

for (i in 1:dim(df_raters)[1]) {
    curr_rater <- df_raters[i,]
        
    mean_raters <- df_d99_mean[curr_rater$fid,] # just so it's set to something for now
    
    #determine current template in order to assign the appropriate mean rater
    if (curr_rater$template == 'd99') {
        mean_raters <- df_d99_mean[curr_rater$fid,]
    } else if (curr_rater$template == 'inia19') {
        mean_raters <- df_inia19_mean[curr_rater$fid,]
    } else if (curr_rater$template == 'macaqueMNI') {
        mean_raters <- df_macaqueMNI_mean[curr_rater$fid,]
    } else if (curr_rater$template == 'nmtv1.3') {
        mean_raters <- df_nmtv1.3_mean[curr_rater$fid,]
    } else if (curr_rater$template == 'yerkes19') {
        mean_raters <- df_yerkes19_mean[curr_rater$fid,]        
    } else {
        # unidentified template, ...
    }
    df_raters[i,]$xdist <- curr_rater$X - mean_raters$X
    df_raters[i,]$ydist <- curr_rater$Y - mean_raters$Y
    df_raters[i,]$zdist <- curr_rater$Z - mean_raters$Z
    curr_coords <- curr_rater[,2:4]
    mean_coords <- mean_raters[,2:4]
    df_raters[i,]$mean_AFLE <- dist3D(curr_coords, mean_coords)
    df_raters[i,]$outlier <- (df_raters[i,]$mean_AFLE > 1.5) # outliers > 1.5mm
}

In [7]:
# summary of findings

# Total
all_templates <- subset(df_raters, session > 0) # ignore session 0 which was from the group tutorial
num_outliers <- sum(subset(all_templates, outlier == TRUE)$outlier)
num_total <- length(all_templates$outlier)
sprintf( "Total: %.2f +/- %.2f mm; Outliers: %d/%d (%.2f%%)",
        mean(all_templates$mean_AFLE), sd(all_templates$mean_AFLE),
        num_outliers, num_total, (num_outliers/num_total)*100 )

# d99
curr_template <- subset(df_raters, session > 0 & template == 'd99')
num_outliers <- sum(subset(curr_template, outlier == TRUE)$outlier)
num_total <- length(curr_template$outlier)
sprintf( "d99: %.2f +/- %.2f mm; Outliers: %d/%d (%.2f%%)",
         mean(curr_template$mean_AFLE), sd(curr_template$mean_AFLE),
         num_outliers, num_total, (num_outliers/num_total)*100 )

# inia19
curr_template <- subset(df_raters, session > 0 & template == 'inia19')
num_outliers <- sum(subset(curr_template, outlier == TRUE)$outlier)
num_total <- length(curr_template$outlier)
sprintf( "inia19: %.2f +/- %.2f mm; Outliers: %d/%d (%.2f%%)",
         mean(curr_template$mean_AFLE), sd(curr_template$mean_AFLE),
         num_outliers, num_total, (num_outliers/num_total)*100 )

# macaqueMNI
curr_template <- subset(df_raters, session > 0 & template == 'macaqueMNI')
num_outliers <- sum(subset(curr_template, outlier == TRUE)$outlier)
num_total <- length(curr_template$outlier)
sprintf( "macaqueMNI: %.2f +/- %.2f mm; Outliers: %d/%d (%.2f%%)",
         mean(curr_template$mean_AFLE), sd(curr_template$mean_AFLE),
         num_outliers, num_total, (num_outliers/num_total)*100 )

# nmtv1.3
curr_template <- subset(df_raters, session > 0 & template == 'nmtv1.3')
num_outliers <- sum(subset(curr_template, outlier == TRUE)$outlier)
num_total <- length(curr_template$outlier)
sprintf( "nmtv1.3: %.2f +/- %.2f mm; Outliers: %d/%d (%.2f%%)",
         mean(curr_template$mean_AFLE), sd(curr_template$mean_AFLE),
         num_outliers, num_total, (num_outliers/num_total)*100 )

# yerkes19
curr_template <- subset(df_raters, session > 0 & template == 'yerkes19')
num_outliers <- sum(subset(curr_template, outlier == TRUE)$outlier)
num_total <- length(curr_template$outlier)
sprintf( "yerkes19: %.2f +/- %.2f mm; Outliers: %d/%d (%.2f%%)",
         mean(curr_template$mean_AFLE), sd(curr_template$mean_AFLE),
         num_outliers, num_total, (num_outliers/num_total)*100 )


[1] "Total: 0.41 +/- 0.31 mm; Outliers: 16/1440 (1.11%)"

[1] "d99: 0.44 +/- 0.34 mm; Outliers: 4/288 (1.39%)"

[1] "inia19: 0.45 +/- 0.39 mm; Outliers: 5/288 (1.74%)"

[1] "macaqueMNI: 0.37 +/- 0.25 mm; Outliers: 0/288 (0.00%)"

[1] "nmtv1.3: 0.35 +/- 0.23 mm; Outliers: 2/288 (0.69%)"

[1] "yerkes19: 0.44 +/- 0.32 mm; Outliers: 5/288 (1.74%)"

In [8]:
# summarize results for each fid and template
summary_all_df <- ddply(df_raters, .(fid), summarize, mean_total=mean(mean_AFLE), sd_total=sd(mean_AFLE), sum_outliers_total=sum(outlier))

summary_temp_df <- ddply(df_raters, .(fid,template), summarize, mean=mean(mean_AFLE), sd=sd(mean_AFLE), sum_outliers=sum(outlier))

# d99
summary_all_df$mean_d99 <- subset(summary_temp_df, template == "d99")$mean
summary_all_df$sd_d99 <- subset(summary_temp_df, template == "d99")$sd
summary_all_df$sum_outliers_d99 <- subset(summary_temp_df, template == "d99")$sum_outliers

# inia19
summary_all_df$mean_inia19 <- subset(summary_temp_df, template == "inia19")$mean
summary_all_df$sd_inia19 <- subset(summary_temp_df, template == "inia19")$sd
summary_all_df$sum_outliers_inia19 <- subset(summary_temp_df, template == "inia19")$sum_outliers

# macaqueMNI
summary_all_df$mean_macaqueMNI <- subset(summary_temp_df, template == "macaqueMNI")$mean
summary_all_df$sd_macaqueMNI <- subset(summary_temp_df, template == "macaqueMNI")$sd
summary_all_df$sum_outliers_macaqueMNI <- subset(summary_temp_df, template == "macaqueMNI")$sum_outliers

# nmtv1.3
summary_all_df$mean_nmtv1.3 <- subset(summary_temp_df, template == "nmtv1.3")$mean
summary_all_df$sd_nmtv1.3 <- subset(summary_temp_df, template == "nmtv1.3")$sd
summary_all_df$sum_outliers_nmtv1.3 <- subset(summary_temp_df, template == "nmtv1.3")$sum_outliers

# yerkes19
summary_all_df$mean_yerkes19 <- subset(summary_temp_df, template == "yerkes19")$mean
summary_all_df$sd_yerkes19 <- subset(summary_temp_df, template == "yerkes19")$sd
summary_all_df$sum_outliers_yerkes19 <- subset(summary_temp_df, template == "yerkes19")$sum_outliers

summary_all_df[,c(-1,-4,-7,-10,-13)] <- round(summary_all_df[,c(-1,-4,-7,-10,-13)],2)
cbind(df_afids,summary_all_df)[-4] # include description so more self-explanatory

write.table(summary_all_df, file = "~/Documents/GitHub/afids-macaca/data/PHASE1_output_afid/PHASE1_template_validation_afid_AFLE.csv", row.names = FALSE, quote = FALSE, sep = ",")

name,description,side,mean_total,sd_total,sum_outliers_total,mean_d99,sd_d99,sum_outliers_d99,mean_inia19,...,sum_outliers_inia19,mean_macaqueMNI,sd_macaqueMNI,sum_outliers_macaqueMNI,mean_nmtv1.3,sd_nmtv1.3,sum_outliers_nmtv1.3,mean_yerkes19,sd_yerkes19,sum_outliers_yerkes19
1,AC,midline,0.15,0.09,0,0.15,0.09,0,0.15,...,0,0.13,0.04,0,0.12,0.09,0,0.19,0.11,0
2,PC,midline,0.16,0.09,0,0.22,0.11,0,0.14,...,0,0.12,0.04,0,0.19,0.09,0,0.15,0.07,0
3,infracollicular sulcus,midline,0.33,0.19,0,0.37,0.22,0,0.42,...,0,0.24,0.09,0,0.22,0.14,0,0.42,0.18,0
4,PMJ,midline,0.24,0.11,0,0.22,0.13,0,0.28,...,0,0.22,0.11,0,0.24,0.13,0,0.25,0.08,0
5,superior interpeduncular fossa,midline,0.24,0.14,0,0.19,0.11,0,0.24,...,0,0.27,0.14,0,0.14,0.06,0,0.33,0.15,0
6,R superior LMS,right,0.37,0.24,0,0.33,0.25,0,0.52,...,0,0.45,0.23,0,0.27,0.14,0,0.28,0.16,0
7,L superior LMS,left,0.40,0.24,0,0.33,0.22,0,0.58,...,0,0.50,0.21,0,0.31,0.12,0,0.29,0.16,0
8,R inferior LMS,right,0.44,0.22,0,0.36,0.18,0,0.47,...,0,0.37,0.18,0,0.58,0.28,0,0.41,0.19,0
9,L inferior LMS,left,0.43,0.21,0,0.40,0.18,0,0.48,...,0,0.41,0.20,0,0.53,0.23,0,0.34,0.21,0
10,culmen,midline,0.43,0.35,2,0.81,0.55,2,0.48,...,0,0.29,0.14,0,0.22,0.15,0,0.34,0.16,0


In [9]:
ddply(df_raters, .(rater), summarize, mean=mean(mean_AFLE), sd=sd(mean_AFLE), sum_outliers=sum(outlier))

rater,mean,sd,sum_outliers
1,0.4110143,0.1899111,0
2,0.3209823,0.2271999,1
3,0.3406045,0.2723814,2
4,0.4441768,0.2520691,0
5,0.5453407,0.5039952,8
6,0.5294000,0.4052402,5


# Post-QC analysis TODO

In [10]:
# versioning info
sessionInfo()

R version 3.6.1 (2019-07-05)
Platform: x86_64-apple-darwin13.4.0 (64-bit)
Running under: macOS Catalina 10.15.2

Matrix products: default
BLAS/LAPACK: /Users/jclau/anaconda3/envs/r-tutorial/lib/R/lib/libRblas.dylib

locale:
[1] en_CA.UTF-8/en_CA.UTF-8/en_CA.UTF-8/C/en_CA.UTF-8/en_CA.UTF-8

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
[1] ggplot2_3.1.1  reshape2_1.4.3 digest_0.6.18  plyr_1.8.4    

loaded via a namespace (and not attached):
 [1] Rcpp_1.0.1       magrittr_1.5     tidyselect_0.2.5 munsell_0.5.0   
 [5] uuid_0.1-2       colorspace_1.4-1 R6_2.4.0         rlang_0.3.4     
 [9] dplyr_0.8.0.1    stringr_1.4.0    tools_3.6.1      grid_3.6.1      
[13] gtable_0.3.0     withr_2.1.2      htmltools_0.3.6  assertthat_0.2.1
[17] lazyeval_0.2.2   tibble_2.1.1     crayon_1.3.4     IRdisplay_0.7.0 
[21] purrr_0.3.2      repr_0.19.2      base64enc_0.1-3  IRkernel_0.8.15 
[25] glue_1.3.1       evaluate_0.13    